In [ ]:
# default_exp core

# Core

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
#hide 
import torch.nn.functional as F
import torch as torch
from functools import partial

In [ ]:
#export
def test():
    '''a test function'''
    print('test')

In [ ]:
test()

test


### Loss Functions

In [ ]:
#export
def leaky_loss(preds, y_true, alpha=0.05):
    '''
    objective function, including negative predictions with factor alpha
    '''
    loss_1 = (F.leaky_relu(preds, alpha).squeeze()*y_true.float()).mean()*(-1)
    loss_1.requires_grad_(True)
    return loss_1

In [ ]:
preds = torch.tensor([-0.5, 0.7, 0.2, -1.5])
y_true = torch.tensor([100., 100., 100., 100.])
assert leaky_loss(preds, y_true) == (-0.5*100*0.05 + 0.7*100 + 0.2*100 + -1.5*100*0.05)/(4*-1)

In [ ]:
y_true_2d = torch.tensor([[100., -90.], [-420., -110], [100.,100.], [-96., 100.]])

In [ ]:
y_true_2d.T.shape

torch.Size([2, 4])

In [ ]:
(F.leaky_relu(preds, 0.5)*y_true_2d.T).mean()*(-1)

tensor(42.0625)

In [ ]:
torch.tensor([0.7, 0.3])[:, None].shape

torch.Size([2, 1])

In [ ]:
(F.leaky_relu(preds, 0.5)*y_true_2d.T)*torch.tensor([2, 0.5])[:, None]

tensor([[ -50.0000, -588.0000,   40.0000,  144.0000],
        [  11.2500,  -38.5000,   10.0000,  -37.5000]])

In [ ]:
((F.leaky_relu(preds, 0.05)*y_true_2d.T)*torch.tensor([2, 0.5])[:, None]).mean()

tensor(-71.2156)

In [ ]:
#export
def leaky_loss_2d(preds, y_true, alpha=0.05, weights=None):
    '''
    objective function, including negative predictions with factor alpha
    weights: target variable weights
    '''
    assert len(y_true.shape)==2, 'y_true needs to be 2d'
     # weight of the first y-value
    prod = (F.leaky_relu(preds, alpha).squeeze()*y_true.float())
    print(prod)
    if weights:
        prod.mul_(torch.tensor(weights)[:, None])
#     print(prod)
    loss_1 = prod.mean()*(-1)
    loss_1.requires_grad_(True)
    return loss_1


In [ ]:
assert torch.allclose(leaky_loss_2d(preds, y_true_2d), (F.leaky_relu(preds, 0.05)*y_true_2d.T).mean()*(-1))

tensor([[  -2.5000, -294.0000,   20.0000,    7.2000],
        [   2.2500,  -77.0000,   20.0000,   -7.5000]])
tensor([[  -2.5000, -294.0000,   20.0000,    7.2000],
        [   2.2500,  -77.0000,   20.0000,   -7.5000]])


In [ ]:
leaky_loss_2d(preds, y_true_2d)

tensor([[  -2.5000, -294.0000,   20.0000,    7.2000],
        [   2.2500,  -77.0000,   20.0000,   -7.5000]])
tensor([[  -2.5000, -294.0000,   20.0000,    7.2000],
        [   2.2500,  -77.0000,   20.0000,   -7.5000]])


tensor(41.4437, requires_grad=True)

In [ ]:
leaky_loss_2d(preds, y_true_2d, weight=[1.,2])

tensor([[  -2.5000, -294.0000,   20.0000,    7.2000],
        [   2.2500,  -77.0000,   20.0000,   -7.5000]])
tensor([[  -2.5000, -294.0000,   20.0000,    7.2000],
        [   4.5000, -154.0000,   40.0000,  -15.0000]])


tensor(49.2250, requires_grad=True)

In [ ]:
leaky_loss_2d(preds, y_true_2d, weight=[1.,2])

tensor([[  -2.5000, -294.0000,   20.0000,    7.2000],
        [   2.2500,  -77.0000,   20.0000,   -7.5000]])
tensor([[  -2.5000, -294.0000,   20.0000,    7.2000],
        [   4.5000, -154.0000,   40.0000,  -15.0000]])


tensor(49.2250, requires_grad=True)

In [ ]:
t=torch.tensor([1., 2.])

In [ ]:
t.mul_(3)

tensor([3., 6.])

In [ ]:
t

tensor([3., 6.])

In [ ]:
(-0.5*100*0.05 + 0.7*100 + 0.2*100 + -1.5*100*0.05)/4

20.0

### Metrics

In [ ]:
#export
def unweighted_profit(preds, y_true, threshold=0):
    '''
    metric, negative predictions ignored, y_true of positive predictions equally weighted
    '''
    m_value = ((preds.squeeze()>threshold).float()*y_true.float()).mean()
    return m_value

In [ ]:
#export
def unweighted_profit_05(preds, y_true, threshold=0.5):
    '''
    metric, negative predictions ignored, y_true of positive predictions equally weighted
    '''
    m_value = ((preds.squeeze()>threshold).float()*y_true.float()).mean()
    return m_value

In [ ]:
assert unweighted_profit(preds, y_true) == (-0.5*100*0 + 1*100 + 1*100 + -1.5*100*0)/(4)

In [ ]:
#export
def weighted_profit(preds, y_true, threshold=0):
    '''
    metric, negative predictions ignored, results weighted by positive predictions
    adding threshold possible
    '''
    loss_1 = ((preds.squeeze()>threshold).float()*(preds.squeeze())*y_true.float()).mean()
    return loss_1

In [ ]:
assert weighted_profit(preds, y_true) == (-0.5*100*0 + 0.7*100 + 0.2*100 + -1.5*100*0)/(4)

### Convenience

In [ ]:
#export
def get_loss_fn(loss_fn_name, **kwargs):
    '''
    wrapper to create a partial with a more convenient __name__ attribute
    '''
    if loss_fn_name == 'leaky_loss':
        assert kwargs.get('alpha', None) is not None, 'need to specify alpha with leaky_loss'
        _loss_fn = partial(leaky_loss, alpha=kwargs['alpha'])
        _loss_fn.__name__ = loss_fn_name
        return _loss_fn
    return None

In [ ]:
assert get_loss_fn('leaky_loss', alpha=0.5)(preds, y_true) == leaky_loss(preds, y_true, alpha=0.5)

In [ ]:
#export
#fastcore.foundations
def is_array(x): return hasattr(x,'__array__') or hasattr(x,'iloc')
def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str) or _is_array(o): return [o]
    if is_iter(o): return list(o)
    return [o]